# eLabFTW - API workshop - Interact using python

## eLabFTW's Python Library: `elabapi-python`

description python sympa
Python is a programming language 

We will now send requests to the API using python, and more precisely a library that has been designed for eLabFTW : elabapi_python.

This library provides a structured approach to sending API requests, following a defined schema. All available methods and their usage are documented in the official API reference.

## Setup the Environment

To ensure a clean and isolated workspace, we will set up a **Python virtual environment** and install the required library using `pip`, as per the documentation suggests.

### Create a Virtual Environment  
Run the following command to create a virtual environment named `venv`:

```bash
python -m venv venv
```

### Activate the Virtual Environment

#### On macOS/Linux:

In [3]:
!python -m venv venv

### Install the required library

Once the virtual environment is activated, install `elabapi-python`:

In [4]:
!pip install elabapi-python

Defaulting to user installation because normal site-packages is not writeable


After installation, your environment is ready to interact with the eLabFTW API. 🚀

📖 **API Documentation:**

🔗 **[doc-elabftw-elabapi](https://doc.elabftw.net/api/elabapi-html/)**

Additionally, example scripts tailored for users can be found in the **examples** folder of the GitHub repository:


📂 **Example Scripts:**

🔗 **[Examples using elabapi-python](https://github.com/elabftw/elabapi-python/tree/master/examples)**

### Sending a Request to the API

To start, we will send a request to the **Info** endpoint, similar to the first example using `curl`. The response should match the output from our earlier `curl` request, confirming that our API authentication and setup are working correctly.

## 🔍 Retrieving Environment Variables

In Python, you can get environment variables by using `os` module. This allows your script to access sensitive variables (such as the key) without having to write them in the script and prevent accidental leakage.

In [5]:
import os

Let us verify that the correct parameters are present:

In [65]:
print(os.getenv("API_URL"))
print(os.getenv("API_KEY"))

https://elab.local:3148/api/v2/
apiKey4Test


## Configuring the eLabAPI-Python Client  

The following code sets up the `elabapi-python` client by providing the necessary API key, host URL, and security settings.  

In [43]:
configuration = elabapi_python.Configuration()

Creates a new configuration object to store API settings.

In [44]:
configuration.api_key["api_key"] = os.getenv("API_KEY")
configuration.api_key_prefix["api_key"] = "Authorization"

Turns off debug mode, meaning the client won’t print detailed request logs.

In [1]:
configuration.host = os.getenv("API_URL")

NameError: name 'os' is not defined

In [47]:
configuration.verify_ssl = False

Disables TLS verification () (not recommended for production, but useful for testing in a development environment).

In [70]:
api_client = elabapi_python.ApiClient(configuration)
api_client.set_default_header(header_name="Authorization", header_value=API_KEY)

- Creates an API client using the configured settings, allowing interaction with the eLabFTW API.
- Explicitly sets the Authorization header for requests.
- This ensures that every API request includes the authentication key automatically.

When making API requests to a server with an **untrusted** or **self-signed SSL certificate**, Python’s `urllib3` library will raise security warnings. The following code disables those warnings:

In [52]:
import urllib3
urllib3.disable_warnings(category=urllib3.exceptions.InsecureRequestWarning)

Now that the API client is configured, we can create an instance of the **Info API client** and send a request to retrieve system information.

In [54]:
infoApi = elabapi_python.InfoApi(api_client)

api_response = infoApi.get_info()

print(api_response)

{'active_users_count': 9,
 'all_users_count': 10,
 'elabftw_version': '5.2.0-alpha',
 'elabftw_version_int': 50200,
 'experiments_count': 326,
 'experiments_timestamped_count': 0,
 'items_count': 232,
 'teams_count': 4,
 'ts_balance': 0,
 'uploads_filesize_sum': 2079,
 'uploads_filesize_sum_formatted': '2.03 KiB'}


## post exp 

Now let's post an experiment

In [60]:
experimentsApi = elabapi_python.ExperimentsApi(api_client)

body={'title':'hi'} 

api_response = experimentsApi.post_experiment(body=body)
print(api_response)

None


# patch exp


# post fichier


# patch ficher


# get le fchier


# delete